In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import glob
from neuropy.utils.mixins.print_helpers import ProgressMessagePrinter


In [3]:
known_data_session_type_dict = {'kdiba':Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'),
                'bapun':Path('R:\data\Bapun\Day5TwoNovel'),
                'rachel':Path('R:\data\Rachel\merged_M1_20211123_raw_phy')}

active_path = known_data_session_type_dict['rachel']

In [12]:
def find_numpy_files(in_directory: Path):
    return list(glob.glob(str(active_path.joinpath('*.np[yz]'))))

def robust_load_npy_file(f):
    if isinstance(f, str):
        f = Path(f)
    if f.is_file():
        dict_rep = None
        try:
            dict_rep = np.load(f, allow_pickle=True)
        except NotImplementedError:
            print("Issue with pickled POSIX_PATH on windows for path {}, falling back to non-pickled version...".format(f))
            temp = pathlib.PosixPath
            pathlib.PosixPath = pathlib.PurePosixPath # Bad hack
            dict_rep = np.load(f, allow_pickle=True)
        try:
            dict_rep = dict_rep.item() # extract the single item if it's a scalar
        except ValueError:
            # print(f'dict_rep not an array of size 1: {dict_rep}')
            pass
        return dict_rep
    else:
        return None


active_numpy_files = find_numpy_files(active_path)
active_numpy_files

['R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\whitening_mat_inv.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\spike_templates.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\channel_shanks.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\spike_clusters.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\similar_templates.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\pc_feature_ind.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\merged_M1_20211123_raw.paradigm.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\spike_times.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\whitening_mat.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\templates.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\channel_map.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\pc_features.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\channel_positions.npy',
 'R:\\data\\Rachel\\merged_M1_20211123_raw_phy\\amplitudes.npy',
 'R:\\data\\Rachel\\merged

In [13]:
numpy_vars = {}
for np_name in active_numpy_files:
    # print(f'{np_name}')
    np_path = Path(np_name)
    with ProgressMessagePrinter(np_name, 'Loading', f'{Path(np_name).stem}'):
        numpy_vars[np_name] = robust_load_npy_file(np_path)
        # numpy_vars[np_name] = np.load(np_name)

numpy_vars

In [21]:
numpy_vars.keys()
{a_path:np.shape(loaded_item) for a_path, loaded_item in numpy_vars.items()}
pd.DataFrame({'file':list(numpy_vars.keys()), 'type':[type(loaded_item) for loaded_item in numpy_vars.values()], 'shape':[np.shape(loaded_item) for loaded_item in numpy_vars.values()]})
# pd.DataFrame({'file':list(numpy_vars.keys()), 'type':[type(loaded_item) for loaded_item in numpy_vars.values()], 'shape':[np.shape(loaded_item) for loaded_item in numpy_vars.values()]})

In [ ]:
# Get dict objects
dict_only_numpy_vars = {a_path:loaded_item for a_path, loaded_item in numpy_vars.items() if isinstance(loaded_item, dict)}
dict_only_numpy_vars